#### Данная тетрадка - предсказания на бейзлайне, т.е. преобразования минимальны, сокращение значений в признаках utm_*, device_* чтобы небыло много ohe-признаков

In [1]:
from warnings import filterwarnings
from datetime import datetime as dt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
import pickle
import pandas as pd
import numpy as np
import gc
from datetime import datetime as dt
filterwarnings('ignore')

In [2]:
datatypes = {'session_id': 'string',
 'client_id': 'string',
 'visit_date': 'string',
 'visit_time': 'string',
 'visit_number': 'int64',
 'utm_source': 'string',
 'utm_medium':'string',
 'utm_campaign':'string',
 'utm_adcontent':'string',
 'utm_keyword':'string',
 'device_category':'string',
 'device_os':'string',
 'device_brand':'string',
 'device_model':'string',
 'device_screen_resolution': 'string',
 'device_browser':'string',
 'geo_country':'string',
 'geo_city':'string',
 'target_event': 'int64'}

df = pd.read_csv("D:\\Libraries\\Projects\\Skillbox.FinalProject\\data\\stage#1.csv", index_col=False, dtype=datatypes )
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

In [4]:
res = df[~((df.duplicated(['session_id'])) & (df['target_event']==0) )]
res = res.sample(n=200000).reset_index(drop=True)
for col in res.columns:
    res[col] = res[col].fillna('unknown')
    
useful_features = [
    'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'device_category',
    'device_brand', 'device_os', 'geo_city', 'target_event'
]
features_to_engineering = ['utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'device_os', 'geo_city']
res = res[useful_features].copy()

In [5]:
best_brands = ['Samsung', 'Xiaomi', 'Huawei', 'POCO']

dev_types = ['mobile', 'tablet']

android_brands = ['Xiaomi', 'Samsung', 'Huawei', 'Vivo', 'LG',
       'BQ', 'OnePlus', 'itel', 'Asus', 'Sony', 'Android', 'OPPO',
       'Nokia', 'Realme', 'Alcatel', 'Infinix', 'Micromax', 'Blackview',
       'Oukitel', 'ZTE', 'Motorola', 'HOMTOM', 'Google', 'Lenovo',
       'China Phone', 'Neffos', 'Tecno', 'DOOGEE', 'Vsmart', 'Meizu', 'Umidigi', 'Leagoo', 'Highscreen', 'Ulefone', 'HTC',
       'Cubot', 'Symphony', 'Vestel', 'Guophone', 'Wileyfox', 'Vertex',
       'POCO', 'TCL', 'SonyEricsson', 'Nomu', 'Prestigio',
       'Philips', 'LeEco', 'DEXP', 'Hisense', 'Vernee', 'Tele2', 'CAT',
       'H96', 'Kyocera', 'Inoi', 'Acer', 'Kingplay', 'Sharp',
       'MTC', 'iNew', 'Karbonn', 'Wiko', 'Gionee', 'Digma',
       'Yota Devices', 'Kata', 'Poptel', 'Flylion', 'AGM', 'Teclast',
       'Fly', 'MLS', 'LeTV', 'Nuu', 'Celkon', 'Chuwi', 'Walton',
       'Haier', 'Coolpad', 'Xiaolajiao', 'Lava', 'General Mobile',
       'Black Fox', 'Condor', 'Razer', 'Microsoft', 'Fujitsu',
       'Alldocube', 'ZOJI', 'Amazon', 'PPTV', 'Jiake', 'Mito', 'Oysters',
       'NOA', 'Caterpillar', 'SenseIT', 'Verizon', 'Komu', 'BLU', 'RCA',
       'Cube', 'AT&T', 'Tonbux', 'Dark', 'Elephone', 'T-Mobile',
       'Smartfren', 'Billion', 'Gome', 'A1', 'MTS', 'Geotel', 'BLUBOO',
       'Ananda', 'MediaTek', 'KingSing', 'Ark', 'Essential',
       'Dragon Touch', 'E&L', 'Fero',  'Wigor', 'Tanix',
       'Mlais', 'InFocus', 'Smartisan', 'Beelink', 'Unihertz', 'Orbic',
       'Honeywell', 'Archos', 'RED', 'Sonim', 'Jiayu', 'Evertek',
       'Leegoog', 'QMobile', 'INUI', 'Artel', 'Irbis', 'China Mobile',
       'Flexymove', 'Olla', 'Krip', 'Star', 'UMI', 'Iris', 'Ellipsis',
       'LTC', 'Motive', 'Advan', 'M-HORSE', 'Winnovo', 'ThL', 'Nomi',
       'Wings Mobile', 'MXQ', 'How', 'Centric', 'Maze', 'Corn',
       'China TVBox', 'Gretel', 'Vertu', 'Evercoss', 'teXet', 'Qbex',
       'Panasonic', 'Tagital', 'Dell', 'Maxvi']

In [6]:
%%time
def foo_os(x):
    if x['device_os']=='unknown':
        if x['device_brand'] in android_brands:
            return 'Android'
        elif (x['device_brand'] == 'Apple') and (x.device_category in dev_types):
            return 'iOS'
    elif x['device_os'] != x['device_os']:
        if (x['device_category'] in dev_types) and (x['device_brand']=='Apple'):
            return 'iOS'
        elif x['device_brand'] in android_brands:
            return 'Android'
    else:
        return x['device_os']
    return x['device_os']
    
res['device_os'] = res.apply(lambda x: foo_os(x), axis=1)
res = res.drop(['device_category', 'device_brand'], axis=1)

CPU times: total: 3.19 s
Wall time: 3.21 s


In [7]:
def change_feature(x):
    grouped = res.groupby([x])['target_event'].mean().to_frame().reset_index()
    res[x]=res[x].apply(lambda a: grouped.loc[grouped[x]==a, 'target_event'].iloc[0])
    res[x]=res[x].astype(np.float64)
    
cat_features = res.select_dtypes(include=['string', 'object', 'bool']).columns.tolist()
for feature in cat_features:
    change_feature(feature)
# change_feature('month')

df_ohed = res.copy()

In [8]:
%%time
# res = res.sample(n=200000).reset_index(drop=True)

print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc
x = df_ohed.drop(['target_event'], axis=1)
y = df_ohed['target_event']
del df_ohed
gc.collect()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

#=======================================================================================================================
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(class_weight='balanced', random_state=42)
# clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(x_train, y_train)

proba_test = clf.predict_proba(x_test)
proba_test = proba_test[:, 1]
proba_train = clf.predict_proba(x_train)
proba_train = proba_train[:, 1]

predicted_train = clf.predict(x_train)
predicted_test = clf.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predicted_train)
score_test  = accuracy_score(y_test, predicted_test)
print('DesicionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predicted_test))
# del clf
# gc.collect()
#=====================================================================
print(f'RandomForestClassifier:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-1, random_state=42)
forest.fit(x_train, y_train)

proba_test = forest.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = forest.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = forest.predict(x_train)
predict_test  = forest.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
# print('\nRandomForestClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))
# del forest
# gc.collect()
#=====================================================================
print(f'\nExtraTreeClassifier:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.ensemble import ExtraTreesClassifier

extree = ExtraTreesClassifier(class_weight='balanced', random_state=42)
extree.fit(x_train, y_train)

proba_test = extree.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = extree.predict_proba(x_train)
proba_train = proba_train[:, 1]

predict_train = extree.predict(x_train)
predict_test  = extree.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
# print('\nExtraTreesClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

print(f'End cell:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')

Start cell: 2024-03-25 13:33:30
DesicionTreeClassifier:
 Качество на тренировочной выборке: 0.5917571428571429
 Качество на тестовой выборке:      0.5770333333333333
 ROc AUC на тренировочной выборке:  0.8112026695459146
 ROC AUC на тестовой выборке:       0.6124802143530567
[[33573 24710]
 [  668  1049]]
RandomForestClassifier:   2024-03-25 13:33:31
 Качество на тренировочной выборке: 0.6241142857142857
 Качество на тестовой выборке:      0.6113333333333333
 ROc AUC на тренировочной выборке:  0.7924089387591929
 ROC AUC на тестовой выборке:       0.6219337762021953
[[35689 22594]
 [  726   991]]

ExtraTreeClassifier:   2024-03-25 13:33:51
 Качество на тренировочной выборке: 0.5917571428571429
 Качество на тестовой выборке:      0.5778833333333333
 ROc AUC на тренировочной выборке:  0.8112026695459146
 ROC AUC на тестовой выборке:       0.6139574320710235
[[33627 24656]
 [  671  1046]]
End cell:   2024-03-25 13:34:05
CPU times: total: 3min 26s
Wall time: 34.9 s


Здесь осуществим подбор гиперпараметров модели

In [9]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.model_selection import GridSearchCV

params_forest = {
    'class_weight': ['balanced'],
    'n_estimators': list(range(25, 45)),
    'max_depth': list(range(5, 10)),
    'min_samples_leaf': list(range(12, 25)),
    'max_features': ['sqrt', 'log2', None]
}


forest = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=forest,
    param_grid=params_forest,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

forest_tuned = RandomForestClassifier(
    class_weight=     'balanced',
    n_estimators=     best_params['n_estimators'],
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features']
)

forest_tuned.fit(x_train, y_train)
proba_test = forest_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = forest_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = forest_tuned.predict(x_train)
predict_test  = forest_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('RandomForestClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-25 13:34:05
Fitting 3 folds for each of 3900 candidates, totalling 11700 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 24, 'n_estimators': 42}
RandomForestClassifier:
 Качество на тренировочной выборке: 0.5104571428571428
 Качество на тестовой выборке:      0.5091
 ROc AUC на тренировочной выборке:  0.7102540145442802
 ROC AUC на тестовой выборке:       0.687854147204204
[[29238 29045]
 [  409  1308]]
CPU times: total: 1min 47s
Wall time: 48min 56s


In [10]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# from sklearn.model_selection import GridSearchCV

params_dtc = {
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'randoms'],
    'max_depth': list(range(5, 15)),
    'min_samples_leaf': list(range(12, 35)),
    'max_features': ['sqrt', 'log2', None]
}


dtc = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=dtc,
    param_grid=params_dtc,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

dtc_tuned = DecisionTreeClassifier(
    class_weight=     'balanced',
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features'],
    criterion=        best_params['criterion'],
    splitter=         best_params['splitter']
)

dtc_tuned.fit(x_train, y_train)
proba_test = dtc_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = dtc_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = dtc_tuned.predict(x_train)
predict_test  = dtc_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('DecisionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-25 14:23:02
Fitting 3 folds for each of 4140 candidates, totalling 12420 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 34, 'splitter': 'best'}
DecisionTreeClassifier:
 Качество на тренировочной выборке: 0.5021857142857142
 Качество на тестовой выборке:      0.5010666666666667
 ROc AUC на тренировочной выборке:  0.6977359294649762
 ROC AUC на тестовой выборке:       0.6740151389734128
[[28748 29535]
 [  401  1316]]
CPU times: total: 1min 7s
Wall time: 2min 28s


In [11]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# from sklearn.model_selection import GridSearchCV

params_extree = {
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy', 'log_loss'],
#     'splitter': ['best', 'randoms'],
    'max_depth': list(range(5, 15)),
    'min_samples_leaf': list(range(12, 35)),
    'max_features': ['sqrt', 'log2', None]
}


extree = ExtraTreesClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=extree,
    param_grid=params_extree,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

extree_tuned = ExtraTreesClassifier(
    class_weight=     'balanced',
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features'],
    criterion=        best_params['criterion']
#     splitter=         best_params['splitter']
)

extree_tuned.fit(x_train, y_train)
proba_test = extree_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = extree_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]

    
predict_train = extree_tuned.predict(x_train)
predict_test  = extree_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('ExtraTreesClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-25 14:25:30
Fitting 3 folds for each of 2070 candidates, totalling 6210 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'max_features': None, 'min_samples_leaf': 22}
ExtraTreesClassifier:
 Качество на тренировочной выборке: 0.5118428571428572
 Качество на тестовой выборке:      0.5096333333333334
 ROc AUC на тренировочной выборке:  0.715783109045451
 ROC AUC на тестовой выборке:       0.6889363839569327
[[29270 29013]
 [  409  1308]]
CPU times: total: 1min 18s
Wall time: 45min 9s


In [12]:
import pickle

In [13]:
forest_model_path = 'models\\forest.pkl'
dtc_model_path = 'models\\dtc.pkl'
extree_model_path = 'models\\extree.pkl'

with open(forest_model_path, 'wb') as f:
    pickle.dump(forest_tuned, f)
    
with open(dtc_model_path, 'wb') as f:
    pickle.dump(dtc_tuned, f)

with open(extree_model_path, 'wb') as f:
    pickle.dump(extree_tuned, f)
